## Hopfield Network


## Introduction

Saw [this video](https://www.youtube.com/watch?v=piF6D6CQxUw) and decided to build one myself.

Generally, A hopfield network is made up of a "grid" of fully connected neurons, each neuron taking the state of other neurons as input and changes it's own state accordingly.

To be more specific.

Given a hopfield network $G(V,E)$:
1. It is Fully connected i.e. $\forall v_i, v_j \in V, \exists e = \{v_i, v_j\}, e \in E$.
2. For vertex v_i, it is assigned a value 1 or -1.
3. We assign a weight matrix $W$ of $|E|$ ($ = |V| \cdot |V|$ when vertex is FC). $W [i,j]$ corresponds to the edgeweight of edge $e[v_i, v_j]$
4. For vertex v_i, the update rule is 
$$

f(\sum_{k: e \{v_k, v_i\} \in E}w_{ki} v_k)\\
f(x)= 
\begin{cases}
    1,& \text{if } x\geq 0\\
    -1,& \text{otherwise}
\end{cases}
$$
5. The vertexs updates asynchronously

Of course, we will flip some of those statements on it's head, which made some of the statements (and implementation of the hopfield network) wierd

## Construction of the class

We will implement the class as we go

In [1]:

# Imports
import numpy as np
from PIL import Image
import time
import os
from IPython.display import clear_output

In [2]:
def print_array(array: np.array): #Temporary, will change later, can only print 2d, binary matrices correctly
    array = array.tolist()
    array = [["□" if x > 0 else "■" for x in row] for row in array]
    for row in array:
        print(' '.join(map(str, row)))
    

class HopNet:
    
    def __init__(self, dims: np.array, weights: np.ndarray = None, init_state: np.ndarray = None, context_dims: int | np.ndarray = None, show_steps = True, verbose = True):   
        # Excemption testing
        if weights is not None and np.array(dims).prod() != weights.shape[0]: # We assign weights between "all pairs" of vertices
            raise ValueError("Mismatch between number of vertices and edges")
        if init_state is not None and dims.shape != init_state.shape:
            raise ValueError("Mismatch between number of vertices and initial state")
        # Assigning values
        self.dims = dims
        self.weights = self.__set_weights(weights)
        self.state = self.__set_state(init_state)
        self.context_window = self.__set_context_dims(context_dims)
        self.verbose = verbose
        self.show_steps = show_steps
    
    def show_state(self):
        print_array(self.state)

    def set_state(self, state: np.ndarray = None):
        self.state = self.__set_state(state)
        
    def update(self, sync = True, max_iter = 10000, wait = 0.1):
        for i in range(max_iter):
            clear_output()
            updated = True
            if sync:
                updated = self.update_sync()
            else:
                updated = self.update_async()
            # if self.show_steps:
            #     self.show_state()
            if not updated:
                self.__print_v("Convergence Reached at iteration {i}".format(i = i))
                break 
            time.sleep(wait)

    def update_async(self):
        prev_state = self.state
        for i in range(self.dims[0]):
            for j in range(self.dims[1]):
                self.state[i,j] = self.__updated_neuron_value(i,j)
                if self.show_steps:
                    clear_output()
                    self.show_state()
                    
        # Check if updated
        if np.array_equal(self.state,prev_state):
            return False
        return True
                
    def update_sync(self):
        new_state = np.zeros(self.dims)
        for i in range(self.dims[0]):
            for j in range(self.dims[1]):
                new_state[i,j] = self.__updated_neuron_value(i,j)
        
        # Check if updated
        if np.array_equal(self.state, new_state):
            return False
        self.state = new_state
        return True
    
    def __updated_neuron_value(self, this_i, this_j):
        this_vertex_i = this_i * self.dims[1] + this_j
        
        start_i = max(0, this_i - self.context_window[0])
        end_i = min(self.state.shape[0], this_i + self.context_window[0] + 1)
        start_j = max(0, this_j - self.context_window[1])
        end_j = min(self.state.shape[1], this_j + self.context_window[1] + 1)

        sum = 0
        for i in range(start_i, end_i):
            for j in range(start_j, end_j):
                if i == this_i and j == this_j:
                    continue
                input_vertex_j = i * self.dims[1] + j
                sum += self.weights[this_vertex_i,input_vertex_j] * self.state[i,j]
        if sum > 0:
            return 1
        else:
            return -1
        
    
    def __print_v(self, msg):
        if self.verbose:
            print(msg)
    
    # Random Weights if not initizalized
    def __set_weights(self, weights):
        if isinstance(weights, np.ndarray):
            if weights.shape[0] != np.array(self.dims).prod():
                raise ValueError("Mismatch between number of vertices and edges")
            return weights
        elif weights is None:
            num_vertices = np.array(self.dims).prod()
            weights = np.random.rand((num_vertices, num_vertices))
        else:
            raise ValueError("Provided Weights must be a numpy array")
    
    # Random start State if not initizalized
    def __set_state(self, init_state):
        if init_state is None:
            state = np.random.randint(0,2,self.dims)
            state[state <= 0] = -1 # Change all 0s to -1s
            return state
        return init_state
    
    def __set_context_dims(self, context_dims):
        ctxt_dims = context_dims
        if context_dims is None:
            ctxt_dims = np.array([self.dims[0], self.dims[1]])
        elif isinstance(ctxt_dims, int):
            ctxt_dims = np.array([ctxt_dims, ctxt_dims])
        else:
            if ctxt_dims[0] > self.dims[0]:
                ctxt_dims[0] = self.dims[0]
            if ctxt_dims[1] > self.dims[1]:
                ctxt_dims[1] = self.dims[1]
        return ctxt_dims
    
    

class HopNetBuilder:
    
    def __init__(self, size: int = None, dims: np.ndarray = None, context_dims: int | np.ndarray = None, verbose = True):
        self.dims = self.__set_dims(size, dims)
        self.memory = [] # List of memories
        self.context_dims = context_dims
        self.verbose = verbose
        
    def add_memory(self, content = None):
        match content:
            case str():
                self.memory.append(Memory(dims=self.dims, image_path=content))
                self.__print_v("Added Memory from image path")
            case np.ndarray:
                self.memory.append(Memory(dims=self.dims, array=content))
                self.__print_v("Added Memory from an array")
            case None:
                self.memory.append(Memory(dims=self.dims))
                self.__print_v("Added a random Memory")
            case _:
                raise ValueError("Must Provide either Image Path (string) or Numpy Array")

    def memory_length(self):
        return len(self.memory)
    
    def memory_show(self, index: int = None):
        if index == None:
            print("There are a total of {num} memories".format(num = len(self.memory)))
            for index_, memory in enumerate(self.memory):
                print("Memory {i}".format(i = index_))
                memory.show()
        else:
            self.memory[index].show()
    
    def memory_delete(self, index: int):
        del self.memory[index]
    
    def memory_clear(self):
        self.memory = []
    
    def __set_dims(self, size, dims):
        match (size, dims):
            case (None, None):
                raise ValueError("Must provide either size or dims")
            case (size, None):
                return np.array([size, size])
            case (None, dims):
                return dims
            case _:
                raise ValueError("Must provide either size or dims, not both")
            
    def build(self, init_state: np.ndarray = None, show_steps = True, verbose = True):
        return HopNet(self.dims, self.__get_weights(), init_state = init_state, context_dims = self.context_dims, show_steps = show_steps, verbose = verbose)
    
    def __get_weights(self):
        num_vertices = np.array(self.dims).prod()
        weights = np.zeros((num_vertices, num_vertices))
        for memory in self.memory:
            weights += np.outer(memory.contents, memory.contents)
        weights = weights / num_vertices
        weights = weights - np.diag(weights) # Set diagonal to 0
        return weights
    
    def __print_v(self, msg):
        if self.verbose:
            print(msg)
            
class Memory:
    
    def __init__(self, dims: np.ndarray, image_path: str = None, array: np.ndarray = None):
        self.dims = dims
        self.contents = self.__set_contents(image_path, array, dims)
        
    def show(self):
        print_array(self.contents)
    
    def __set_contents(self, image_path, array, dims):
        match (image_path, array):
            # No arguements Provided, return random memory
            case (None, None):
                return self.__set_random_memory(dims)
            case (image_path, None):
                return self.__set_image_memory(image_path, dims)
            case (None, array):
                return self.__set_array_memory(array, dims)
            case _:
                raise ValueError("Must provide either an image path or array")
    
    def __set_random_memory(self, dims):
        memory = np.random.randint(0,2,dims)
        memory[memory <= 0] = -1 # Change all 0s to -1s
        return memory
    
    def __set_image_memory(self, image_path, dims):
        image = Image.open(image_path)
        image = image.resize(dims)
        image = image.convert("L")
        memory = np.array(image)
        memory = (memory >= 128).astype(float)
        memory[memory == 0] = -1 # Change all 0s to -1s
        return memory
        
    def __set_array_memory(self, array: np.ndarray, dims):
        if np.array(np.shape(array)) != dims:
            raise ValueError("Provided matrix have wrong dimensions")
        memory = array
        memory[memory >= 0] = -1
        memory[memory < 0] = 1
        return memory

## Testing the network

In [3]:
builder = HopNetBuilder(dims = np.array([12,12]))

builder.add_memory("image/test.png")
builder.add_memory("image/test1.png")

Added Memory from image path
Added Memory from image path


In [4]:
builder.memory_show()

There are a total of 2 memories
Memory 0
■ ■ ■ ■ ■ ■ □ □ □ □ □ □
■ ■ ■ ■ ■ ■ □ □ □ □ □ □
■ ■ ■ ■ ■ ■ □ □ □ □ □ □
■ ■ ■ ■ ■ ■ □ □ □ □ □ □
■ ■ ■ ■ ■ ■ □ □ □ □ □ □
■ ■ ■ ■ ■ ■ □ □ □ □ □ □
□ □ □ □ □ □ ■ ■ ■ ■ ■ ■
□ □ □ □ □ □ ■ ■ ■ ■ ■ ■
□ □ □ □ □ □ ■ ■ ■ ■ ■ ■
□ □ □ □ □ □ ■ ■ ■ ■ ■ ■
□ □ □ □ □ □ ■ ■ ■ ■ ■ ■
□ □ □ □ □ □ ■ ■ ■ ■ ■ ■
Memory 1
■ ■ ■ □ □ □ □ □ □ ■ ■ ■
■ ■ ■ ■ □ □ □ □ ■ ■ ■ ■
■ ■ ■ ■ ■ □ □ ■ ■ ■ ■ ■
□ ■ ■ ■ ■ ■ □ ■ ■ ■ ■ □
□ □ ■ ■ ■ ■ ■ ■ ■ ■ □ □
□ □ □ ■ ■ ■ ■ ■ ■ □ □ □
□ □ □ □ ■ ■ ■ ■ ■ □ □ □
□ □ □ ■ ■ ■ ■ ■ ■ ■ □ □
□ □ ■ ■ ■ ■ ■ ■ ■ ■ ■ □
□ ■ ■ ■ ■ □ □ ■ ■ ■ ■ ■
■ ■ ■ ■ □ □ □ □ □ ■ ■ ■
■ ■ ■ □ □ □ □ □ □ □ ■ ■


In [5]:
network = builder.build()

In [18]:
network.set_state()
network.show_state()
time.sleep(1)
network.update(sync = False, max_iter = 1000, wait = 0.1)

□ □ □ ■ ■ ■ ■ ■ ■ □ □ □
□ □ □ □ ■ ■ ■ ■ □ □ □ □
□ □ □ □ □ ■ ■ □ □ □ □ □
■ □ □ □ □ □ ■ ■ ■ ■ ■ ■
■ ■ □ □ □ □ ■ ■ ■ ■ ■ ■
■ ■ ■ □ □ □ □ □ □ ■ ■ ■
■ ■ ■ ■ □ □ □ □ □ ■ ■ ■
■ ■ ■ □ □ □ □ □ □ □ ■ ■
■ ■ □ □ □ □ □ □ □ □ □ ■
■ □ □ □ □ ■ ■ □ □ □ □ □
□ □ □ □ ■ ■ ■ ■ ■ □ □ □
□ □ □ ■ ■ ■ ■ ■ ■ ■ □ □
Convergence Reached at iteration 0


In [19]:
network.show_state()

□ □ □ ■ ■ ■ ■ ■ ■ □ □ □
□ □ □ □ ■ ■ ■ ■ □ □ □ □
□ □ □ □ □ ■ ■ □ □ □ □ □
■ □ □ □ □ □ ■ ■ ■ ■ ■ ■
■ ■ □ □ □ □ ■ ■ ■ ■ ■ ■
■ ■ ■ □ □ □ □ □ □ ■ ■ ■
■ ■ ■ ■ □ □ □ □ □ ■ ■ ■
■ ■ ■ □ □ □ □ □ □ □ ■ ■
■ ■ □ □ □ □ □ □ □ □ □ ■
■ □ □ □ □ ■ ■ □ □ □ □ □
□ □ □ □ ■ ■ ■ ■ ■ □ □ □
□ □ □ ■ ■ ■ ■ ■ ■ ■ □ □
